## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
import os
file_to_load = os.path.join("..","Weather_Database","cities.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,64.3833,-173.3000,47.19,91,100,2.53,overcast clouds
1,1,Mahebourg,MU,-20.4081,57.7000,70.09,60,75,11.50,broken clouds
2,2,Kavaratti,IN,10.5669,72.6420,81.77,83,99,17.16,overcast clouds
3,3,Labuan,MY,5.2767,115.2417,77.36,94,20,2.30,light rain
4,4,Shelburne,CA,44.0787,-80.2041,63.23,80,100,7.02,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Select minimum desired temperature? "))
max_temp = float(input("Select maximum desired temperature? "))

Select minimum desired temperature? 70
Select maximum desired temperature? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mahebourg,MU,-20.4081,57.7000,70.09,60,75,11.50,broken clouds
3,3,Labuan,MY,5.2767,115.2417,77.36,94,20,2.30,light rain
7,7,Atuona,PF,-9.8000,-139.0333,77.68,72,21,24.07,few clouds
9,9,Kapaa,US,22.0752,-159.3190,72.12,87,100,11.36,overcast clouds
10,10,Jambi,ID,-1.6000,103.6167,75.38,96,25,1.63,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
682,682,North Branch,US,45.5113,-92.9802,71.58,88,84,8.19,broken clouds
685,685,San Cristobal,VE,7.7669,-72.2250,77.18,58,45,7.72,scattered clouds
687,687,Mitsamiouli,KM,-11.3847,43.2844,76.66,70,26,6.35,scattered clouds
689,689,Chichester,GB,50.8367,-0.7800,76.05,64,100,3.00,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,70.09,broken clouds,-20.4081,57.7000,
3,Labuan,MY,77.36,light rain,5.2767,115.2417,
7,Atuona,PF,77.68,few clouds,-9.8000,-139.0333,
9,Kapaa,US,72.12,overcast clouds,22.0752,-159.3190,
10,Jambi,ID,75.38,scattered clouds,-1.6000,103.6167,
17,Tutubigan,PH,75.40,light rain,11.8397,125.0614,
19,Bambous Virieux,MU,70.05,broken clouds,-20.3428,57.7575,
21,Casper,US,72.72,overcast clouds,42.8666,-106.3131,
22,Dingle,PH,73.11,overcast clouds,10.9995,122.6711,
25,Nyuksenitsa,RU,75.20,overcast clouds,60.4128,44.2397,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    params['location'] = f"{latitude},{longitude}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_data['results'][0]['name']
    except:
        print("No hotel found. Skipping ... ")
        
        

No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 
No hotel found. Skipping ... 


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
hotel_df.head()
# hotel_df[hotel_df['Hotel Name'] == ""]


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,70.09,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
3,Labuan,MY,77.36,light rain,5.2767,115.2417,Tiara Labuan Hotel
7,Atuona,PF,77.68,few clouds,-9.8000,-139.0333,Villa Enata
9,Kapaa,US,72.12,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Jambi,ID,75.38,scattered clouds,-1.6000,103.6167,ASTON Jambi Hotel & Conference Center


In [12]:
!pwd

/Users/johnwhendricks/Desktop/classFolder/World-Weather-Analysis/Vacation_Search


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))